<a href="https://colab.research.google.com/github/m0hit-kumar/RASA/blob/main/CollabNotebook/TASC_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!unzip Corrected_data.zip


Archive:  Corrected_data.zip
   creating: Corrected_data/
  inflating: Corrected_data/BombBlast(categorised).json  
  inflating: Corrected_data/Earthquake(categorised).json  
  inflating: Corrected_data/flood(categorised).json  
  inflating: Corrected_data/tornado(categorised).json  
  inflating: Corrected_data/tsnuami (categorised).json  
  inflating: Corrected_data/Volcano(categorised).json  
  inflating: Corrected_data/war(categorised).json  


In [5]:
import json
import os

all_data = []
for filename in os.listdir('Corrected_data/'):
    if filename.endswith(".json"):
        file_path = os.path.join('Corrected_data/', filename)
        with open(file_path, 'r') as file:
            data = json.load(file)
            all_data.extend(data)

with open("Data.json", 'w') as outfile:
    json.dump(all_data, outfile, indent=4)

All JSON files have been combined into 'Data.json'


In [76]:
import pandas as pd
import numpy as np
data=pd.read_json("Data.json")

In [77]:
data.head()

,text,intent
0,Mr O'Flynn said he was at the finish of the ev...,Miscellaneous
1,"""While I know how sore it must be, let none of...",Miscellaneous
2,"Nancy Ward, director of the California Governo...",Miscellaneous
3,Corpses of soldiers lay strewn on the streets ...,Miscellaneous
4,"The forecaster added: ""The next few days will ...",floods


In [78]:
data["intent"].unique()

array(['Miscellaneous', 'floods', 'Miscellenous', 'Miscellaenous',
       'Miscellneous', 'BombBlast', 'Micscellaneous', 'Micsellaneous',
       'War', 'Micellaneous', 'miscellaneous', 'Miscellaeous',
       'Earthquake', nan, 'Volcano', 'Tornado', 'Miscellanous',
       'Miscelleneous'], dtype=object)

In [81]:
# checlking for nan value
data['intent'].fillna('misc', inplace=True)

data.loc[data["intent"].str.startswith("m"),"intent"]="misc"
data.loc[data["intent"].str.startswith("M"),"intent"]="misc"

In [82]:
data["intent"].unique()

array(['misc', 'floods', 'BombBlast', 'War', 'Earthquake', 'Volcano',
       'Tornado'], dtype=object)

TypeError: ignored

In [41]:
for i in data["intent"]:
  ifdata["intent"]=

SyntaxError: ignored